To run the array ephys workflow, we need to properly set up the DataJoint configuration. The configuration will be saved in a file called `dj_local_conf.json` on each machine and this notebook walks you through the process.


**The configuration only needs to be set up once**, if you have gone through the configuration before, directly go to [02-workflow-structure](02-workflow-structure-optional.ipynb).

# Set up configuration in root directory of this package

As a convention, we set the configuration up in the root directory of the workflow package and always starts importing datajoint and pipeline modules from there.

In [1]:
import os
os.chdir('..')

In [2]:
pwd

'/Users/davidgodinez/Desktop/Datajoint/workflow-array-ephys'

In [2]:
import datajoint as dj

# Configure database host address and credentials

Now let's set up the host, user and password in the `dj.config` global variable

In [4]:
import getpass
dj.config['database.host'] = 'localhost'
dj.config['database.user'] = 'davidgodinez'
dj.config['database.password'] = getpass.getpass() # enter the password securily

········


You should be able to connect to the database at this stage.

In [5]:
dj.conn()

Connecting davidgodinez@localhost:3306


DataJoint connection (connected) davidgodinez@localhost:3306

# Configure the `custom` field in `dj.config` for the element-array-ephys

The major component of the current workflow is the [DataJoint Array Ephys Element](https://github.com/datajoint/element-array-ephys). Array Ephys Element requires configurations in the field `custom` in `dj.config`:

## Database prefix

Giving a prefix to schema could help on the configuration of privilege settings. For example, if we set prefix `neuro_`, every schema created with the current workflow will start with `neuro_`, e.g. `neuro_lab`, `neuro_subject`, `neuro_ephys` etc.

The prefix could be configurated as follows in `dj.config`:

In [6]:
dj.config['custom'] = {'database.prefix': 'neuro_'}

## Root directories for raw ephys data and kilosort 2 processed results

+ `ephys_root_data_dir` field indicates the root directory for the **ephys raw data** from SpikeGLX or OpenEphys (e.g. `*imec0.ap.bin`, `*imec0.ap.meta`, `*imec0.lf.bin`, `imec0.lf.meta`) or the **clustering results** from kilosort2 (e.g. `spike_times.npy`, `spike_clusters.npy`). The root path typically **do not** contain information of subjects or sessions, all data from subjects/sessions should be subdirectories in the root path.

In the example dataset downloaded with [this instruction](00-data-download-optional.ipynb), `/tmp/test_data` will be the root

```
/tmp/test_data/
- subject6
    - session1
        - towersTask_g0_imec0
        - towersTask_g0_t0_nidq.meta
        - towersTask_g0_t0.nidq.bin
```

In [1]:
# If there is only one root path.
dj.config['custom']['ephys_root_data_dir'] = '/Users/davidgodinez/Desktop/Datajoint/workflow-array-ephys/tmp/test_data'
# If there are multiple possible root paths:
dj.config['custom']['ephys_root_data_dir'] = ['/tmp/test_data']

NameError: name 'dj' is not defined

In [8]:
dj.config

{   'connection.charset': '',
    'connection.init_function': None,
    'custom': {   'database.prefix': 'neuro_',
                  'ephys_root_data_dir': ['/tmp/test_data']},
    'database.host': 'localhost',
    'database.password': 'password1',
    'database.port': 3306,
    'database.reconnect': True,
    'database.use_tls': None,
    'database.user': 'davidgodinez',
    'display.limit': 7,
    'display.show_tuple_count': True,
    'display.width': 14,
    'enable_python_native_blobs': True,
    'fetch_format': 'array',
    'loglevel': 'INFO',
    'safemode': True}

+ In the database, every path for the ephys raw data is **relative to this root path**. The benefit is that the absolute path could be configured for each machine, and when data transfer happens, we just need to change the root directory in the config file.
+ The workflow supports **multiple root directories**. If there are multiple possible root directories, specify the `ephys_root_data_dir` as a list.
+ The root path(s) are **specific to each machine**, as the name of drive mount could be different for different operating systems or machines.
+ In the context of the workflow, all the paths saved into the database or saved in the config file need to be in the **POSIX standards** (Unix/Linux), with `/`. The path conversion for machines of any operating system is taken care of inside the elements.

# Save the configuration as a json file

With the proper configurations, we could save this as a file, either as a local json file, or a global file.

In [9]:
dj.config.save_local()

In [10]:
ls

CONTRIBUTING.md           docker-compose-test.yaml  tests/
Dockerfile                env/                      tmp/
LICENSE                   images/                   user_data/
README.md                 notebooks/                venv/
StandardConfig.json       requirements.txt          workflow_array_ephys/
apt_requirements.txt      requirements_test.txt
dj_local_conf.json        setup.py


Local configuration file is saved as `dj_local_conf.json` in the root directory of this package `workflow-array-ephys`. Next time if you change your directory to `workflow-array-ephys` before importing datajoint and the pipeline packages, the configurations will get properly loaded.

If saved globally, there will be a hidden configuration file saved in your root directory. The configuration will be loaded no matter where the directory is.

In [24]:
# dj.config.save_global()

# Next Step

After the configuration, we will be able to review the workflow structure with [02-workflow-structure-optional](02-workflow-structure-optional.ipynb).